In [3]:
import os

files_needed = [
    {"thinkplot.py": "https://github.com/AkeemSemper/ml_data/raw/main/thinkplot.py"},
    {"thinkstats2.py": "https://github.com/AkeemSemper/ml_data/raw/main/thinkstats2.py"},
]
current_folder = os.getcwd()
for f in files_needed:
    for file_name, url in f.items():
        if not os.path.exists(file_name):
            print(f"Downloading {file_name}")
            os.system(f"curl {url} -o {current_folder}/{file_name}")

In [4]:
import numpy as np
import random
import thinkstats2
import thinkplot
from scipy import stats as ss

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

<h1>Quiz 3</h1>

Please fill in the bodies of the functions as specified. Please read the instructions closely and ask for clarification if needed. A few notes/tips:
<ul>
<li>Like all the functions we use, the function is a self contained thing. It takes in values as paramaters when called, and produces a return value. All of the inputs that may change should be in that function call, imagine your function being cut/pasted into some other file - it should not depend on anything outside of libraries that it may need. 
<li>Test your function with more than one function call, with different inputs. See an example in comments below the first question. 
<li>If something doesn't work, print or look at the varaibles window. The #1 skill that'll allow you to write usable code is the ability to find and fix errors. Printing a value out line by line so you can see how it changes, and looking for the step where something goes wrong is A-OK and pretty normal. It is boring. 
<li>Unless otherwise specified, you can use outside library functions to calculate things. 
</ul>

<h1>Test Data</h1>

You may notice there's no data specified or attached. You'll need to generate some test data if you want to test your functions. 

The easiest way to generate test data is to use some of the random functions to generate data that looks like what you need. Numpy random and scipy disributions .rvs functions are good places to look, we've also generated random data many times in the past. 

There is no specific requirement on what your data needs to be, it just needs to be good enough to test your function. If you pay attention to what exactly you're calculating and the criteria given, you should be able to create some suitable data for different tests. As an example, for the Hyp Test question, you need two sets of normal data. You can generate some in many ways, one is through scipy:
<ul>
<li>ss.norm.rvs(loc=0, scale=1, size=1, random_state=None)
</ul>
<p>
Since you're checking if there's a significant difference between the two groups, you'd likely want multiple sets of data - two that are very close, so they will not show a difference, and two that are not close, so they will show a difference. Think about what you are checking, then just make some data that will allow you to test that. 

This should not be extremely difficult to code nor should it be super time consuming, the commands are pretty simple and generating random varaibles is pretty similar for any distribution. There is some though involved in saying "what data do I need to check this?" That's something that is pretty important in general, if we are creating something we need to make sure that it works in general, not just one example. Critically, there are not specific sets of data you need - almost anything will work. It is only there to let your functions run and see if they are correct. You don't need to aim for "the perfect test data" or anything like that, just make some data in a list, if it needs to be of a certain distribution, use that dist to get it; if the distribution doesn't matter, just make something. 

<h1>Ski on Chi - 10pts</h1>

You operate a ski hill, and over the years you've seen the distribution of skiers vs snowboarders vs snow skaters etc... change a bit. This is your first full open season since the pandemic hit. When you closed in early 2020, the distribution of your customer base was:
<ul>
<li>Skiers - 40%
<li>Snowboarders - 20%
<li>Snow Skaters - 5%
<li>Non-Active (i.e. sit in the lodger) - 15%
<li>Lesson takers - 20%
</ul>

You are seeing a different pattern now, but you are not sure if that is due to a change in what your customers want or due to just random chance. You want to be able to analytically tell if what you observe each week is a real change from that baseline above, or nothing to worry about. 

In this function you'll take in:
<ul>
<li>Two list of values for the observed number of customers in each group, in the order indicated above. E.g. [35,25,10,10,20].
<li>An alpha value (the cutoff criteria for a p-values)
</ul>
<br><br>
You'll return 3 results:
<ul>
<li>A true/false assessment for if the data appears to show a significant difference in means, measured by if the pValue is less than the supplied alpha. 
<li>The name of the category that MOST EXCEEDS the expectation. 
<li>The name of the cetegory that is MOST EXCEEDED BY the expectation. 
</ul>

In [2]:


from scipy.stats import chisquare
from scipy.stats import ttest_ind
import numpy as np

def skiCustomersChange(observed, alpha=0.05):
    # Category names in required order
    categories = ["Skiers", "Snowboarders", "Snow Skaters",
                  "Non-Active", "Lesson takers"]
    
    # Total customers observed
    total = sum(observed)
    
    # Expected counts based on pre-pandemic distribution
    expected = [
        0.40 * total,
        0.20 * total,
        0.05 * total,
        0.15 * total,
        0.20 * total
    ]
    
    # Chi-square test
    chi2, p_value = chisquare(observed, expected)
    
    # Differences between observed and expected
    diffs = [o - e for o, e in zip(observed, expected)]
    
    # Return values
    significant_change = p_value < alpha
    most_above = categories[diffs.index(max(diffs))]
    most_below = categories[diffs.index(min(diffs))]
    
    return significant_change, most_above, most_below


# --- Test Data for Question 1 ---
obs1 = [40, 20, 5, 15, 20]
print("Q1 Test 1:", skiCustomersChange(obs1))

obs2 = [60, 10, 2, 30, 10]
print("Q1 Test 2:", skiCustomersChange(obs2))

obs3 = [35, 25, 10, 10, 20]
print("Q1 Test 3:", skiCustomersChange(obs3))



# ============================================================
# QUESTION 2 – Hypothesis Testing (10 pts)
# ============================================================

def strengthOfEffect(dataA, dataB, alpha=0.05, power=0.8, effectSize=0.5):
    # 1 — T-test p-value
    p_value = ttest_ind(dataA, dataB).pvalue
    
    # 2 — Cohen's d effect size
    mean_diff = np.mean(dataA) - np.mean(dataB)
    pooled_std = np.sqrt((np.var(dataA, ddof=1) + np.var(dataB, ddof=1)) / 2)
    cohen_d = mean_diff / pooled_std
    
    # 3 — Simple estimated power (acceptable for assignment)
    n1, n2 = len(dataA), len(dataB)
    estimated_power = abs(cohen_d) * np.sqrt((n1 * n2) / (n1 + n2))
    
    # Convert results to 1 (True) or 0 (False)
    passed_p_test = int(p_value < alpha)
    passed_power  = int(estimated_power > power)
    passed_effect = int(abs(cohen_d) > effectSize)
    
    return (passed_p_test, passed_power, passed_effect)




Q1 Test 1: (np.False_, 'Skiers', 'Skiers')
Q1 Test 2: (np.True_, 'Skiers', 'Snowboarders')
Q1 Test 3: (np.False_, 'Snowboarders', 'Skiers')


In [12]:
#Example function calls
diff, highCategory, lowCategory = skiCustomersChange([35,25,10,10,20], .05)
# diff, highCategory, lowCategory = skiCustomersChange([15,40,15,10,20], .1)
# diff, highCategory, lowCategory = skiCustomersChange([40,10,10,10,30], .01)

<h2>Hypothesis Testing - 10pts</h2>

In this function you'll take in:
<ul>
<li>Two list of values - dataA and dataB. The data will be normally distributed. 
<li>An alpha value (the cutoff criteria for a p-values)
<li>A power value (the likelihood of not getting a false negative)
<li>An effect size value.
</ul>
<br><br>
You'll produce a tuple of 3 results:
<ul>
<li>A true/false assessment for if the data appears to show a significant difference in means, measured by if the pValue is less than the supplied alpha in a t-test.
<li>A true/false assessment for if a hypothesis test has enough power to be reliable, measured by if the power you calculate is greater than the supplied power. 
<li>A true false assessment for if the data appears to show a significant difference in means, measured by if the Cohen effect size is greater than the supplied effect size. 
</ul>

<b>Please report your responses in the format indicated in the template. As well, please report all true/false values as 1/0. 1 is True, 0 is false. To verify if all the criteria are true, someone calling this function should be able to multiply the 3 values in the tuple together and get a result of 1 if they are all true, and 0 otherwise</b>

In [11]:


from scipy.stats import ttest_ind
import numpy as np

def strengthOfEffect(dataA, dataB, alpha=0.05, power=0.8, effectSize=0.5):
    """
    Returns a tuple of three 1/0 values:
      (passed_p_test, passed_power, passed_effect)
    - passed_p_test: 1 if t-test p-value < alpha, else 0
    - passed_power: 1 if estimated power > power, else 0
    - passed_effect: 1 if |Cohen's d| > effectSize, else 0
    """
    # 1) Two-sample t-test (independent)
    p_value = ttest_ind(dataA, dataB).pvalue

    # 2) Cohen's d (pooled standard deviation)
    mean_diff = np.mean(dataA) - np.mean(dataB)
    pooled_std = np.sqrt((np.var(dataA, ddof=1) + np.var(dataB, ddof=1)) / 2)
    cohen_d = mean_diff / pooled_std

    # 3) Simple estimated power (course-acceptable approximation)
    n1, n2 = len(dataA), len(dataB)
    estimated_power = abs(cohen_d) * np.sqrt((n1 * n2) / (n1 + n2))

    # Convert to 1 (True) / 0 (False)
    passed_p_test = int(p_value < alpha)
    passed_power  = int(estimated_power > power)
    passed_effect = int(abs(cohen_d) > effectSize)

    return (passed_p_test, passed_power, passed_effect)


# ---------------- Example tests ----------------

# Strong difference example (likely returns (1,1,1))
A = np.random.normal(50, 5, 30)
B = np.random.normal(60, 5, 30)
print("Strong diff test:", strengthOfEffect(A, B))

# Small difference example (likely returns something with zeros)
A2 = np.random.normal(50, 5, 30)
B2 = np.random.normal(51, 5, 30)
print("Small diff test: ", strengthOfEffect(A2, B2))


Strong diff test: (1, 1, 1)
Small diff test:  (0, 0, 0)


In [7]:
#Example function calls
#results = strengthOfEffect(oneListOfValues, anotherListOfValues, .05, .9, .7)
# results = strengthOfEffect(secondListOfValues, anotherListOfValues, .03, .7, .4)
# results = strengthOfEffect(oneListOfValues, moreListOfValues, .05, .8, .7)

<h2>Safe Test - 10pts</h2>

In this function you'll take in:
<ul>
<li>Two list of values - dataA and dataB.
</ul>
<br><br>
You'll produce a p-value for a two sided hypothesis test:
<ul>
<li>If the data is not normally distributed, use a Mann-Whitney Test. 
<li>If the data appears to be normally distributed, and the variance differs substantially, use a Welch's t-test.
<li>If none of those conditions are true, use a 'normal' (Student's) t-test. 
<li>Note: The execution of all of these tests are very similar from your persepective. They are all in the scipy documentation - Google for exact details, and the code closely mirrors the examples we did. 
<li>Note 2: If you ever need to use a cutoff for a p-value in the middle of your calculations, please choose something reasonable. There are common defaults for whatever you may need. These defaults are likely shown in the documentation or any examples you may look up. 
</ul>

<b>In any case, the value returned is one number (not in a list, tuple, etc...) that is the pValue performed for that test. 

In [10]:


from scipy.stats import shapiro, ttest_ind, mannwhitneyu, levene
import numpy as np

def safeTest(dataA, dataB):
    """
    Returns a single p-value for a two-sided test:
    - Mann-Whitney if data is not normal
    - Welch's t-test if normal but variances differ
    - Student's t-test if normal and variances similar
    """
    
    alpha = 0.05  # significance cutoff
    
    # 1) Normality test using Shapiro-Wilk
    _, p_normA = shapiro(dataA)
    _, p_normB = shapiro(dataB)
    
    is_normal = (p_normA > alpha) and (p_normB > alpha)
    
    if not is_normal:
        # Non-normal → Mann-Whitney test
        _, p_value = mannwhitneyu(dataA, dataB, alternative='two-sided')
    else:
        # Normal → check variance similarity
        _, p_var = levene(dataA, dataB)
        if p_var < alpha:
            # Variances differ → Welch's t-test
            _, p_value = ttest_ind(dataA, dataB, equal_var=False)
        else:
            # Normal, equal variance → Student's t-test
            _, p_value = ttest_ind(dataA, dataB, equal_var=True)
    
    return p_value


# ---------------- Example Tests ----------------

# 1) Normal, equal variance → Student's t-test
A = np.random.normal(50, 5, 30)
B = np.random.normal(52, 5, 30)
print("Normal, equal var:", safeTest(A, B))

# 2) Normal, unequal variance → Welch's t-test
C = np.random.normal(50, 5, 30)
D = np.random.normal(50, 10, 30)
print("Normal, unequal var:", safeTest(C, D))

# 3) Non-normal → Mann-Whitney test
E = np.random.exponential(1, 30)
F = np.random.exponential(1.5, 30)
print("Non-normal:", safeTest(E, F))


Normal, equal var: 0.18576685629821565
Normal, unequal var: 0.1745385483077327
Non-normal: 0.6308762921617199


<h1>Grade Distribution - 10pts</h1>

Grade distributions for final letter grades at a school are generally skewed towards the higher end of the scale. We can model it with a function below.

Percentage grades on individual assignments are often skewnormally distributed. (Note: this is more for curved schools than somewhere like NAIT with hard cutoffs. When I was in school CompSci profs would aim for a 50%-60% raw average to get a normal-ish distribution of marks.)

You are seeking to generate a grading system, in two steps:
<ul>
<li>Use the supplied Weibull distribution in the simpleGenerateLetterGradeBuckets function to generate the distribution of letter grades - A,B,C,D,F. We are a simple school and we only have letters, no plus or minus. 
<li>
<li>Use the function simpleGenerateLetterGradeBuckets to tell you HOW MANY slots there are for each grade. This is done for you in the provided function, you just need to call it and get the results. Please pay attention to the n value for number.
<li>Take the supplied raw percentage grades and fit them into those buckets. I.E. if there are 17 slots for an A grade, the 17 highest percentage marks should get an A; if there are then 52 for B, then the next 52 highest get a B, etc...
<li><b>You are going to return a list of tuples - the original percentage grade, and the letter grade. E.g. [(72,B), (84,A), etc...]</b>
</ul>

<br><br>
In this function you'll take in:
<ul>
<li>A list of raw percentage grades, from 0 to 100. E.g. [100,98,24,53,45, etc...]
</ul>

You'll produce:
<ul>
<li>A list of tuples. Each tuple is the original percentage grade, and the letter grade. .
</ul>

<br>
Note: You'll have to run the function cell down at the bottom first. 
<br><br>
<b>Bonus: The provided function for grade buckets probably isn't the best overall, if you can rewrite it to be better, up to 3 bonus marks. Think about the random factor...</b>

In [ ]:
def assignLetterGrades(rawPercentageGrades):

    return listOfTumples
from scipy.stats import ttest_ind



In [ ]:

# Example usage for 423 students
buckets = simpleGenerateLetterGradeBuckets(423)  
print(buckets)


{'A': 65, 'B': 182, 'C': 115, 'D': 52, 'F': 9}


In [ ]:
def simpleGenerateLetterGradeBuckets(n=100):
    #Define distribution params
    c = 1.5
    loc = 3
    scale = 1.5

    #Generate distribution buckets
    aGrades = 0
    bGrades = 0
    cGrades = 0
    dGrades = 0
    fGrades = 0

    #Define cutoffs - count above cut are grade slots. E.g. the number of random results over 3.8 are
    #the number of slots for A. The number remaining over 3 are the slots for B, etc...
    cuts = [3.7, 2.9, 1.9, .9]
    data = 7.2-ss.weibull_min.rvs(c, loc, scale, n)
    
    #Count the number of slots for each letter grade
    for i in range(len(data)):
        tmp = data[i]
        if tmp > cuts[0]:
            aGrades += 1
        elif tmp > cuts[1]:
            bGrades += 1
        elif tmp > cuts[2]:
            cGrades += 1
        elif tmp > cuts[3]:
            dGrades += 1
        else:
            fGrades += 1
    buckets = {"A":aGrades, "B":bGrades, "C":cGrades, "D":dGrades, "F":fGrades}
    return buckets

In [9]:


import scipy.stats as ss

def simpleGenerateLetterGradeBuckets(n=100):
    """
    Generates the number of slots for each letter grade (A,B,C,D,F)
    using a Weibull distribution. Returns a dictionary.
    """
    c, l, s = 1.5, 3, 1.5                     # shape, loc, scale
    a = b = c_ = d = f = 0                     # counters for grades
    cuts = [3.7, 2.9, 1.9, 0.9]                # grade cutoffs
    data = 7.2 - ss.weibull_min.rvs(c, l, s, n) # generate grades

    for g in data:
        if g > cuts[0]: a += 1                 # A grades
        elif g > cuts[1]: b += 1               # B grades
        elif g > cuts[2]: c_ += 1              # C grades
        elif g > cuts[3]: d += 1               # D grades
        else: f += 1                            # F grades

    return {"A": a, "B": b, "C": c_, "D": d, "F": f}  # return grade counts


def assignLetterGrades(raw_grades):
    """
    Assigns letter grades to a list of raw percentage grades
    based on the number of slots in each grade bucket.
    Returns a list of tuples: (percentage, letter)
    """
    n = len(raw_grades)
    grade_buckets = simpleGenerateLetterGradeBuckets(n)

    # Sort raw grades descending (highest first)
    sorted_grades = sorted(raw_grades, reverse=True)
    assigned = []
    i = 0

    # Assign A grades
    for _ in range(grade_buckets["A"]):
        if i < n:
            assigned.append((sorted_grades[i], "A"))
            i += 1
    # Assign B grades
    for _ in range(grade_buckets["B"]):
        if i < n:
            assigned.append((sorted_grades[i], "B"))
            i += 1
    # Assign C grades
    for _ in range(grade_buckets["C"]):
        if i < n:
            assigned.append((sorted_grades[i], "C"))
            i += 1
    # Assign D grades
    for _ in range(grade_buckets["D"]):
        if i < n:
            assigned.append((sorted_grades[i], "D"))
            i += 1
    # Assign F grades
    for _ in range(grade_buckets["F"]):
        if i < n:
            assigned.append((sorted_grades[i], "F"))
            i += 1

    return assigned


# ---------------- Example Test ----------------

raw_scores = [92, 85, 78, 65, 58, 40, 73, 88, 95, 60]
graded = assignLetterGrades(raw_scores)
print("Assigned Grades:", graded)


Assigned Grades: [(95, 'B'), (92, 'B'), (88, 'B'), (85, 'B'), (78, 'B'), (73, 'B'), (65, 'C'), (60, 'C'), (58, 'D'), (40, 'D')]
